# Training Tensorflow model on Google Cloud Vertex AI

In [13]:
!sudo chown -R jupyter:jupyter /home/jupyter/Sentiment-Analysis-of-Amazon-Reviews

In [14]:
import os

In [15]:
ENV_VARS = {'PROJECT': 'text-analysis-323506',
            'REGION': 'us-central1',
            'BUCKET': 'text-analysis-323506',
            'MACHINE_TYPE': 'n1-highmem-4',
            'REPLICA_COUNT': 1,
            'EXECUTE_IMAGE_URI': 'us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-8:latest',
            'LOCAL_PACKAGE_PATH': '/home/jupyter/Sentiment-Analysis-of-Amazon-Reviews/',
            'PYTHON_MODULE': 'detectors.tf_gcp.trainer.task'}

In [16]:
for key, value in ENV_VARS.items():
    os.environ[key] = str(value)

In [17]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Submit Training job to Google Cloud Ai Platform

In [18]:
%%bash

# Move latest config file to google storage bucket
gsutil -m cp -r ../../config gs://$BUCKET

# Specify a unique name for the job.
JOBNAME=sentiment_analysis_$(date -u +%y%m%d_%H%M%S)
echo $REGION $JOBNAME

# Submit training job
gcloud ai custom-jobs create \
    --region=$REGION \
    --display-name=$JOBNAME \
    --worker-pool-spec=machine-type=$MACHINE_TYPE,replica-count=$REPLICA_COUNT,executor-image-uri=$EXECUTE_IMAGE_URI,local-package-path=$LOCAL_PACKAGE_PATH,python-module=$PYTHON_MODULE
    --args=--train-config=gs://$BUCKET/config/config.yaml

us-west1 sentiment_analysis_210912_123115
jobId: sentiment_analysis_210912_123115
state: QUEUED


Copying file://../../config/config.yaml [Content-Type=application/octet-stream]...
Copying file://../../config/.ipynb_checkpoints/config-checkpoint.yaml [Content-Type=application/octet-stream]...
/ [2/2 files][  3.0 KiB/  3.0 KiB] 100% Done                                    
Operation completed over 2 objects/3.0 KiB.                                      
Job [sentiment_analysis_210912_123115] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe sentiment_analysis_210912_123115

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs sentiment_analysis_210912_123115


In [ ]:
# Running TensorBoard
# Open the port you are supposed to use (usually 6006) in Google cloud vm instance
# info -> https://docs.bitnami.com/google/faq/administration/use-firewall/
# Run tensorboard using following command
# tensorboard --logdir <path/to/log/directory> --bind_all
# Ex:
# tensorboard --logdir gs://text-analysis-323506/train_results/tensorboard --bind_all